In [6]:
from icecream import ic
import numpy as np
import rp
import torch
import torch.nn as nn
from source.stable_diffusion import StableDiffusion
from easydict import EasyDict
from source.learnable_textures import (LearnableImageFourier,
                                       LearnableImageFourierBilateral,
                                       LearnableImageRaster,
                                       LearnableImageRasterBilateral,
                                       LearnableTexturePackFourier,
                                       LearnableTexturePackRaster)

In [2]:
if 's' not in dir():
    model_name="CompVis/stable-diffusion-v1-4"
    gpu='cuda:1'
    s=StableDiffusion(gpu,model_name)
device=s.device

[INFO] sd.py: loading stable diffusion...please make sure you have run `huggingface-cli login`.


Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

[INFO] sd.py: loaded stable diffusion!


In [20]:
@rp.monkey_patch(StableDiffusion)
def get_text_embedding(self, prompt: str)->torch.Tensor:
    assert isinstance(prompt, str)

    # Tokenize text and get embeddings
    text_input = self.tokenizer(
        [prompt],
        padding='max_length', 
        max_length=self.tokenizer.model_max_length,
        truncation=True, 
        return_tensors='pt'
    ).input_ids.to(self.device)

    with torch.no_grad():
        text_embeddings = self.text_encoder(text_input).last_hidden_state
        
    assert len(text_embeddings)==1
    text_embedding = text_embeddings[0]

    assert text_embedding.shape == (77, 768)

    return text_embedding


s.get_text_embedding("GEI")


tensor([[-0.3884,  0.0229, -0.0522,  ..., -0.4899, -0.3066,  0.0675],
        [-1.2061, -0.5991,  1.5715,  ..., -1.5062,  0.2366, -0.3361],
        [ 0.4694, -0.4625,  1.2672,  ...,  0.1284,  0.1081, -1.1094],
        ...,
        [-0.3598, -0.5160,  0.3769,  ...,  0.7312, -0.5609, -1.3857],
        [-0.3500, -0.5058,  0.3600,  ...,  0.7610, -0.5657, -1.3863],
        [-0.3219, -0.4319,  0.4588,  ...,  0.7126, -0.5477, -1.4207]],
       device='cuda:1')

In [3]:
def initial_noise():
    return torch.randn(4,64,64).to(device)

In [4]:
prompt = 'big juicy hamburger with cheese and tomato and lettuice. Sesame seed bun. Advertisement beautiful dlsr hdr bokeh. ' 
embeddings = s.get_text_embeddings(prompt)

AttributeError: 'StableDiffusion' object has no attribute 'get_text_embeddings'

In [ ]:
latent = initial_noise()

In [ ]:
def display_latent(latent):
    image = s.decode_latent(latent)
    image = rp.as_numpy_image(image)
    rp.display_image(image)

In [ ]:
display_latent(latent)

In [ ]:
def ddim_step(latent, embedding, t):
    noise = torch.randn_like(latent)
    
    

In [ ]:
for t in range(1,1000,1000//50):
    
    